In [2]:
import findspark

In [3]:
findspark.init('/home/leonardo/spark-3.1.1-bin-hadoop2.7')

In [4]:
import pytest

In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql import functions as F

In [7]:
from pyspark.sql.functions import col

In [8]:
sc = SparkContext.getOrCreate()

In [9]:
sqlContext = HiveContext(sc)

In [10]:
m_destipointeraccion = sqlContext.read.parquet("./inputs caso practico/m_destipointeraccion");
h_interaccioncajero = sqlContext.read.parquet("./inputs caso practico/h_interaccioncajero");

#### GENERACION DE UNIVERSO

In [11]:
#PASO 1
#CREAR UNIVERSO INTERACCIONES VALIDAS
columns1 = ["tipinteraccion", "mtointeraccionsol"]
TP_UNIV_INTERACCION = h_interaccioncajero.where("flgvalida = 'S'").select(*columns1)

In [12]:
TP_UNIV_INTERACCION.show(5)

+--------------+-----------------+
|tipinteraccion|mtointeraccionsol|
+--------------+-----------------+
|        101000|         160.0000|
|        101000|        2000.0000|
|        101000|         100.0000|
|        481040|         290.0000|
|        101000|         100.0000|
+--------------+-----------------+
only showing top 5 rows



In [13]:
#PASO 2
#CREAR UNIVERSO DE MAESTRA TIPO INTERACCIONES VALIDAS PARA CANAL 1002
columns2 = ["tipinteraccion", "destipinteraccion",]
TP_UNIV_TIPO_INTERACCION = h_interaccioncajero.where("codcanal = '1002'").select(*columns2)

In [14]:
TP_UNIV_TIPO_INTERACCION.show(5)

+--------------+--------------------+
|tipinteraccion|   destipinteraccion|
+--------------+--------------------+
|        101000|Retiro cuentas SA...|
|        101000|Retiro cuentas SA...|
|        101000|Retiro cuentas SA...|
|        481040|Transferencia a t...|
|        101000|Retiro cuentas SA...|
+--------------+--------------------+
only showing top 5 rows



In [15]:
#PASO 3
#CRUCE DE TABLAS TEMPORALES
TP_UNIVERSOS_INTERACCION = TP_UNIV_INTERACCION.alias("a") \
                           .join(TP_UNIV_TIPO_INTERACCION.alias("b"), \
                           TP_UNIV_INTERACCION.tipinteraccion == TP_UNIV_TIPO_INTERACCION.tipinteraccion, \
                           how='left').select("a.tipinteraccion", "a.mtointeraccionsol", "b.destipinteraccion")

In [16]:
TP_UNIVERSOS_INTERACCION.show(5)

+--------------+-----------------+--------------------+
|tipinteraccion|mtointeraccionsol|   destipinteraccion|
+--------------+-----------------+--------------------+
|        101000|         160.0000|Retiro cuentas SA...|
|        101000|         160.0000|Retiro cuentas SA...|
|        101000|         160.0000|Retiro cuentas SA...|
|        101000|         160.0000|Retiro cuentas SA...|
|        101000|         160.0000|Retiro cuentas SA...|
+--------------+-----------------+--------------------+
only showing top 5 rows



In [17]:
#PASO 4
#AGRUPACION Y CREACION DE NUEVOS REGISTROS
M_UNIVERSOS_INTERACCION = TP_UNIVERSOS_INTERACCION.groupBy("tipinteraccion", "destipinteraccion") \
                          .agg(F.count(col("tipinteraccion")), F.sum(col("mtointeraccionsol"))) \
                          .withColumnRenamed("tipinteraccion", "INTERACTION_CODE") \
                          .withColumnRenamed("destipinteraccion", "INTERACTION_DESCRIPTION") \
                          .withColumnRenamed("count(tipinteraccion)", "INTERACTION_NUMBER") \
                          .withColumnRenamed("sum(mtointeraccionsol)", "INTERACTION_AMOUNT")

In [34]:
M_UNIVERSOS_INTERACCION.show()

+----------------+-----------------------+------------------+------------------+
|INTERACTION_CODE|INTERACTION_DESCRIPTION|INTERACTION_NUMBER|INTERACTION_AMOUNT|
+----------------+-----------------------+------------------+------------------+
|          101000|   Retiro cuentas SA...|              5644|      2781330.0000|
|          102000|   Retiro cuentas IM...|                 4|         1200.0000|
|          482040|   Transferencia a t...|                 1|          600.0000|
|          291000|   Deposito a cuenta...|                49|        33670.0000|
|          481040|   Transferencia a t...|                36|        14940.0000|
|          441080|   Deposito a cuenta...|                 1|          400.0000|
+----------------+-----------------------+------------------+------------------+



In [39]:
M_UNIVERSOS_INTERACCION.write.parquet("/home/leonardo/Desktop/Code/Practice/Python/SparkDespliegue/Scripts/M_UNIVERSOS_INTERACCION")